<a href="https://colab.research.google.com/github/golesuman/66-daysofdatachallenge/blob/main/Day60/fastapiwithPytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install colabcode
!pip install fastapi
from colabcode import ColabCode
from fastapi import FastAPI
cc=ColabCode(port=12000,code=False)

In [ ]:
from fastapi import  FastAPI,File
from fastapi.datastructures import UploadFile
import torch
# from model import alexnet,predict
model=torch.load('model.pth')
app=FastAPI()

import torch, torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import os
import io
from PIL import Image
image_transforms = { 
    'train': transforms.Compose([
        transforms.RandomResizedCrop(size=(256,256), scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}


alexnet = models.alexnet(pretrained=True)
for param in alexnet.parameters():
    param.requires_grad = False
alexnet.classifier[6] = nn.Linear(4096, 6)
alexnet.classifier.add_module("7", nn.LogSoftmax(dim = 1))

loss_func = nn.NLLLoss()
optimizer = optim.Adam(alexnet.parameters())
model=torch.load('/content/model.pth')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
def predicts(model, test_image_name):
    '''
    Function to predict the class of a single test image
    Parameters
        :param model: Model to test
        :param test_image_name: Test image

    '''
    
    transform = image_transforms['test']

    test_image = Image.open(test_image_name)
    plt.imshow(test_image)
    
    test_image_tensor = transform(test_image)

    if torch.cuda.is_available():
        test_image_tensor = test_image_tensor.view(1, 3, 224, 224).cuda()
    else:
        test_image_tensor = test_image_tensor.view(1, 3, 224, 224)
    
    with torch.no_grad():
        model.eval()
        # Model outputs log probabilities
        out = model(test_image_tensor)
        ps = torch.exp(out)
        topk, topclass = ps.topk(6, dim=1)
        topk=torch.Tensor.cpu(topk)
        topclass=torch.Tensor.cpu(topclass)
        for i in range(6):
            print("Predcition", i+1, ":", idx_to_class[topclass.numpy()[0][i]], ", Score: ", topk.numpy()[0][i])


@app.get('/')
async def main():
    return {"name": "Suman"}


@app.post('/predict')
async def predict(file: UploadFile = File(...)):
    # content=file.file.read()
    # image=Image.open(io.BytesIO(content)).convert('L')
    content=file.file.read()
    image=Image.open(io.BytesIO(content)).convert('L')
    # output=predict(image)
    # return output
    transform = image_transforms['test']

    test_image = Image.open(image)
    plt.imshow(test_image)
    
    test_image_tensor = transform(test_image)

    if torch.cuda.is_available():
        test_image_tensor = test_image_tensor.view(1, 3, 224, 224).cuda()
    else:
        test_image_tensor = test_image_tensor.view(1, 3, 224, 224)
    
    with torch.no_grad():
        model.eval()
        # Model outputs log probabilities
        out = model(test_image_tensor)
        ps = torch.exp(out)
        topk, topclass = ps.topk(6, dim=1)
        topk=torch.Tensor.cpu(topk)
        topclass=torch.Tensor.cpu(topclass)
        for i in range(6):
            return {"Predcition": idx_to_class[topclass.numpy()[0][i]],
                    "Score:": topk.numpy()[0][i]}




In [ ]:
cc.run_app(app=app)